### 1. 安装HuggingFace 并下载模型到本地

In [ ]:
!pip install huggingface-hub -Uqq
!pip install -U sagemaker

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_model_path = Path("./LLM_chatglm2_model")
local_model_path.mkdir(exist_ok=True)
model_name = "THUDM/chatglm2-6b"
commit_hash = "b259b27320263629b0afccef134c54028233673d"

In [ ]:
snapshot_download(repo_id=model_name, revision=commit_hash, cache_dir=local_model_path)

### 2. 把模型拷贝到S3为后续部署做准备

In [ ]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [ ]:
s3_model_prefix = "LLM-RAG/workshop/LLM_chatglm2_model"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/LLM_chatglm2_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

In [ ]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [ ]:
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117"
)

#中国区需要替换为下面的image_uri
# inference_image_uri = (
#     f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.21.0-deepspeed0.8.3-cu117"
# )

print(f"Image going to be used is ---- > {inference_image_uri}")

In [ ]:
!mkdir -p LLM_chatglm2_deploy_code

In [ ]:
%%writefile LLM_chatglm2_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os

from transformers import pipeline, AutoModel, AutoTokenizer

def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_location, trust_remote_code=True)
   
    model = AutoModel.from_pretrained(model_location, trust_remote_code=True).half().cuda()
    
    # model.requires_grad_(False)
    # model.eval()
    
    return model, tokenizer


model = None
tokenizer = None
generator = None


def handle(inputs: Input):
    global model, tokenizer
    if not model:
        model, tokenizer = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = data["inputs"]
    params = data["parameters"]
    history = data["history"]
    
    # chat(tokenizer, query: str, history: List[Tuple[str, str]] = None, 
    # max_length: int = 2048, num_beams=1, do_sample=True, top_p=0.7, 
    # temperature=0.95, logits_processor=None, **kwargs)
    response, history = model.chat(tokenizer, input_sentences, history=history, **params)
    
    result = {"outputs": response, "history" : history}
    return Output().add_as_json(result)

In [ ]:
print(f"option.s3url ==> s3://{bucket}/{s3_model_prefix}/")

#### Note: option.s3url 需要按照自己的账号进行修改, 可以拷贝上一个cell的输出

In [ ]:
%%writefile LLM_chatglm2_deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.s3url = s3://sagemaker-us-west-2-106839800180/LLM-RAG/workshop/LLM_chatglm2_model/

#### 注意: 必须把transformers升级到4.27.1以上，否则会出现 [Issue344](https://github.com/THUDM/ChatGLM-6B/issues/344)

如果是中国区建议添加国内的pip镜像,如下代码所示
```
%%writefile LLM_chatglm_deploy_code/requirements.txt
-i https://pypi.tuna.tsinghua.edu.cn/simple
transformers==4.28.1
```

In [ ]:
%%writefile LLM_chatglm2_deploy_code/requirements.txt
transformers==4.28.1

In [ ]:
!rm model.tar.gz
!cd LLM_chatglm2_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf model.tar.gz LLM_chatglm2_deploy_code

In [ ]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

### 4. 创建模型 & 创建endpoint

In [ ]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base(f"chatglm2") #Note: Need to specify model_name
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

In [ ]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

#Note: ml.g4dn.2xlarge 也可以选择
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.2xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 15*60,
        },
    ],
)
endpoint_config_response

In [ ]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

#### 持续检测模型部署进度

In [ ]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

### 5. 模型测试

In [ ]:
%%time
import json
import boto3

smr_client = boto3.client("sagemaker-runtime")

parameters = {
  "max_length": 2048,
  "temperature": 0.01,
  "num_beams": 1, # >1可能会报错，"probability tensor contains either `inf`, `nan` or element < 0"； 即使remove_invalid_values=True也不能解决
  "do_sample": False,
  "top_p": 0.7,
  "logits_processor" : None,
  # "remove_invalid_values" : True
}

In [ ]:
prompts1 = """你是技术领域的智能问答机器人AIBot，请严格根据反括号中的资料提取相关信息，回答用户的各种问题
```
作业调度工具：Crontab、Airflow、TaskCTL、Moia、Oozie\n\n(2).ETL流程的设计\n\nETL是将数据从源端经过抽取（extract）、转换（transform）、加载（load）至目的端的一个数据流动过程；对于数据仓库的建设，ETL是其中一个比较\n\n重要的环节，通常会根据以下步骤来完成ETL的设计：\n\n1.业务场景的分析\n\n2.源数据的抽取策略及数据的转换规则\n\n3.数据流程的控制\n\n4.数据质量校验和ETL作业监控\n\n(3).数据模型的设计\n\n数据模型是指使用使用实体、属性及其关系对企业运营和逻辑规则进行统一的定义、编码和命名；可以通过对实体和实体之间关系的定义和描述，来表达具体业务之间的关系；数据仓库模型是数据模型中针对特定的数据仓库应用系统的一种特定的数据模型。\n\n1.数据仓库数据模型的作用：\n\n11_文档资料\n\n12_版本上线\n\n13_数据资产\n\n14_服务维护\n\n15_项目\n\n16_AI业务\n\n17_测试相关\n\n18_问题跟踪\n\nE编辑\n\nW关注\n\nS分享\n\n\n\n工具\n\n技术中心\n\n数据智能部\n\n数据智能部 主页\n\n10_参考资料\n\n浅谈数据仓库体系\n\n转至元数据结尾\n\n\n\n\n\nCreated and last modified by  龙佟佟 on 十一月 15, 2018\n\n转至元数据起始\n\n数据仓库(Data Warehouse) 是一个面向主题的(Subject-Oriented) 、集成的( Integrate ) 、相对稳定的(Non-Volatile ) 、反映历史变化( Time-Variant) 的数据集合用于支持管理决策。\n\n一、数据仓库设计方法论\n\n2.ETL任务调度信息、输入输出：由于数据中心的作业都是采用crontab工具调度kettle作业，但任务调度信息不好获取，目前采用解析kettle日志的方式来获取跑批任务的调度信息且信息存储在mysql数据库的t99_sys_etl_job_information表中；后续ETL作业迁移到airflow上时任务调度信息可以直接在airflow的元数据库中获取（可视化展示 Neo4j）。\n\n3.表依赖映射关系 ：目前采用人工维护的方式并存储到mysql数据库的t99_sys_etl_table_base_info表中；后续将完善相关信息（表数据大小，数据热度，更新频率等）。\n\n4.数据仓库的模型定义：数据中心的数据模型不多，目前也只是采用人工手动维护的方式来管理的；由于模型都是通过SQL的方式进行数据逻辑加工的，后续可以采用\n\n作业调度工具：Crontab、Airflow、TaskCTL、Moia、Oozie\n\n(2).ETL流程的设计\n\nETL是将数据从源端经过抽取（extract）、转换（transform）、加载（load）至目的端的一个数据流动过程；对于数据仓库的建设，ETL是其中一个比较\n\n重要的环节，通常会根据以下步骤来完成ETL的设计：\n\n1.业务场景的分析\n\n2.源数据的抽取策略及数据的转换规则\n\n3.数据流程的控制\n\n4.数据质量校验和ETL作业监控\n\n(3).数据模型的设计\n\n数据模型是指使用使用实体、属性及其关系对企业运营和逻辑规则进行统一的定义、编码和命名；可以通过对实体和实体之间关系的定义和描述，来表达具体业务之间的关系；数据仓库模型是数据模型中针对特定的数据仓库应用系统的一种特定的数据模型。\n\n1.数据仓库数据模型的作用：
```
用户: 数据仓库构建有哪些作业调度工具?
AIBot:"""
response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters,
                "history" : []
            }
            ),
            ContentType="application/json",
        )

response_model['Body'].read().decode('utf8')

In [ ]:
prompt="""你是技术领域的智能问答机器人AIBot，请严格根据反括号中的资料提取相关信息，回答用户的各种问题
```
\n11_文档资料\n\n12_版本上线\n\n13_数据资产\n\n14_服务维护\n\n15_项目\n\n16_AI业务\n\n17_测试相关\n\n18_问题跟踪\n\nE编辑\n\nW关注\n\nS分享\n\n\n\n工具\n\n技术中心\n\n数据智能部\n\n数据智能部 主页\n\n10_参考资料\n\n浅谈数据仓库体系\n\n转至元数据结尾\n\n\n\n\n\nCreated and last modified by  龙佟佟 on 十一月 15, 2018\n\n转至元数据起始\n\n数据仓库(Data Warehouse) 是一个面向主题的(Subject-Oriented) 、集成的( Integrate ) 、相对稳定的(Non-Volatile ) 、反映历史变化( Time-Variant) 的数据集合用于支持管理决策。\n\n一、数据仓库设计方法论\n\n作业调度工具：Crontab、Airflow、TaskCTL、Moia、Oozie\n\n(2).ETL流程的设计\n\nETL是将数据从源端经过抽取（extract）、转换（transform）、加载（load）至目的端的一个数据流动过程；对于数据仓库的建设，ETL是其中一个比较\n\n重要的环节，通常会根据以下步骤来完成ETL的设计：\n\n1.业务场景的分析\n\n2.源数据的抽取策略及数据的转换规则\n\n3.数据流程的控制\n\n4.数据质量校验和ETL作业监控\n\n(3).数据模型的设计\n\n数据模型是指使用使用实体、属性及其关系对企业运营和逻辑规则进行统一的定义、编码和命名；可以通过对实体和实体之间关系的定义和描述，来表达具体业务之间的关系；数据仓库模型是数据模型中针对特定的数据仓库应用系统的一种特定的数据模型。\n\n1.数据仓库数据模型的作用：\n\n每种实现方法都有利弊，通常情况是两种的组合；因为数据仓库在开始构建时一般是用自底向上的方法进行的。当在使用阶段性数据仓库项目模型来构建业务范围架构中的一系列数据集市时，可以一个 接一个地集成不同业务主题领域中的数据集市，从而形成设计良好的业务数据仓库体系结构。\n\n二、数据仓库的搭建(传统型数仓)\n\n数据仓库是一个过程而不是一个项目，一般情况下可以按照以下四个步骤去搭建一套数据仓库；\n\n需求/现状 --> 架构 --> 实现 --> 测试/发布\n\n1.需求\n\n在搭建数仓前都会有前期的需求调研和分析准备工作，并从各个部门的角度定义计划和需求；其中需求可以大致分为以下三类：\n\n(1)业务需求:一般指数据仓库最终用户的信息需求；\n\n(2)合规需求:一般是企业上报的数据需求；\n\n(3)ETL需求:指数仓建设过程中的数据需求，如数据质量评估（可以更好的了解数据内容、结构和质量）。\n\n2.架构\n\n(1)当前架构\n\n数据仓库通常采用分层体系架构,一般简单的数据仓库会分为三层，贴源层 ,历史存储层,数据模型层：\n\n<1>贴源层：把源库中的数据同步到数据仓中来，一般都会与源体系数据保持一致（一方面能够保证我们数据仓库的数据来源是真实存在的，另一方面在出了问题方便进行核对)。\n\n对于贴源层抽取源系统的数据，一般有两种方式：1.全量抽取  2.增量抽取\n\n<2>历史层：同步贴源层数据并保存历史变化的数据;该层通常都会采用FS-LDM主题域模型来对表进行划分。\n\n对于历史层来说，一般有三种存储方式:1.增量切片 2.全量切片 3.增量拉链 （具体表设计需要根据业务，存储和使用来综合评估）
```
用户: 数据仓库搭建的需求有哪几类
AIBot: """

response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompt,
                "parameters": parameters,
                "history" : []
            }
            ),
            ContentType="application/json",
        )

response_model['Body'].read().decode('utf8')

#### 清除模型Endpoint和config

In [ ]:
!aws sagemaker delete-endpoint --endpoint-name chatglm2-2023-06-27-06-53-41-986-endpoint

In [ ]:
!aws sagemaker delete-endpoint-config --endpoint-config-name chatglm2-2023-06-27-06-53-41-986-config

In [ ]:
!aws sagemaker delete-model --model-name chatglm2-2023-06-27-06-53-41-986